# Лабораторная работа №1
## Выполнил студент группы БФИ2001

### Задание №1
Hello, World!

In [1]:
print("Hello, World!")

Hello, World!


### Задание №2
Написать генератор случайных матриц(многомерных), который принимает
опциональные параметры **m**, **n**, **min_limit**, **max_limit**,
где **m** и **n** указывают размер матрицы, а **min_lim** и **max_lim** - минимальное и максимальное значение для генерируемого числа.


По умолчанию при отсутствии параметров принимать следующие значения:
1. **m** = 50
1. **n** = 50
1. **min_lim** = -250
1. **max_lim** = 1019

In [2]:
from typing import Generator
from random import randint


def matrix_generator(
        m: int = 50,
        n: int = 50,
        min_limit: int = -250,
        max_limit: int = 1019) -> Generator:
    for _ in range(m):
        yield (randint(min_limit, max_limit) for _ in range(n))

### Задание №3
Реализовать методы сортировки строк числовой матрицы в соответствии с заданием. Оценить время работы каждого алгоритма сортировки и сравнить его со временем стандартной функции сортировки. Испытания проводить на сгенерированных матрицах.


Методы:
1. Выбором
1. Вставкой
1. Обменом
1. Шелла
1. Турнирная
1. Быстрая
1. Пирамидальная

In [3]:
from typing import Any, Callable, Union

In [4]:
%%timeit
def selection_sort(a: list[int]) -> list[int]:
    arr_len: int = len(a)

    for i, elem in enumerate(a):
        minimum = min(range(i, arr_len), key=a.__getitem__)
        a[i], a[minimum] = a[minimum], elem

    return a

520 ns ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
%%timeit
def insertion_sort(a: list[int]) -> list[int]:
    for i in range(1, len(a)):
        value = a[i]
        j = i - 1
        while j >= 0 and a[j] > value:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = value

    return a

495 ns ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
%%timeit
def stooge_sort(a: list[int], i: int = 0, j: Union[int, None] = None) -> list[int]:
    if j is None:
        j = len(a) - 1

    if a[i] > a[j]:
        a[i], a[j] = a[j], a[i]

    if j - i > 1:
        one_third = (j - i + 1) // 3
        stooge_sort(a, i, j - one_third)
        stooge_sort(a, i + one_third, j)
        stooge_sort(a, i, j - one_third)

    return a

1.01 µs ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
def shell_sort(a: list[int]) -> list[int]:
    mid: int = len(a) // 2

    while mid:
        for i, elem in enumerate(a):
            while i >= mid and a[i - mid] > elem:
                a[i] = a[i - mid]
                i -= mid
            a[i] = elem

        if mid == 2:
            mid = 1
        else:
            mid //= 2

    return a

496 ns ± 14.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
%%timeit
def tournament_sort(a: list[int]) -> list[int]:
    a_len: int = len(a)
    heap = PriorityHeap()
    winners: list[int] = []
    losers: list[int] = []

    while heap.size < heap.max_size and heap.size < a_len:
        elem = a[heap.size]
        if winners and winners[-1] > elem:
            losers.append(elem)
        else:
            heap.max_heap_insert(a[heap.size])

    for i in range(heap.size, a_len):
        elem: int = a[i]

        if winners and winners[-1] > elem:
            losers.append(elem)
        else:
            max_elem: int = heap.pop_max()

            if not winners:
                winners.append(max_elem)
            elif winners[-1] <= max_elem:
                winners.append(max_elem)
            else:
                losers.append(max_elem)

            heap.max_heap_insert(elem)

    while heap.size > 0:
        max_elem: int = heap.pop_max()

        if not winners:
            winners.append(max_elem)
        elif winners[-1] <= max_elem:
            winners.append(max_elem)
        else:
            losers.append(max_elem)

    if not losers:
        return winners
    else:
        winners2: list[int] = tournament_sort(losers)

        i: int = 0
        j: int = 0
        winners1_len: int = len(winners)
        winners2_len: int = len(winners2)

        winners_res: list[int] = []

        while i < winners1_len and j < winners2_len:
            if winners[i] < winners2[j]:
                winners_res.append(winners[i])
                i += 1
            else:
                winners_res.append(winners2[j])
                j += 1

        while i < winners1_len:
            winners_res.append(winners[i])
            i += 1

        while j < winners2_len:
            winners_res.append(winners2[j])
            j += 1

        return winners_res

534 ns ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
class PriorityHeap:
    def __init__(self) -> None:
        self.cache: list[int] = []
        self.size: int = 0
        self.max_size = 32

    def pop_max(self) -> Any:
        res = self.cache[0]
        self.size -= 1
        self.cache[0] = self.cache[self.size]
        self.cache.pop(self.size)
        self.iter_heapify(0)
        return res

    def max_heap_insert(self, key: int):
        self.cache.append(-10000000000000)
        self.insert_key(key, self.size)
        self.size += 1


    def insert_key(self, value: int, i: int):
        self.cache[i] = value
        parent = self.parent(i)
        while i > 0 and self.cache[parent] > self.cache[i]:
            self.cache[parent], self.cache[i] = self.cache[i], self.cache[parent]
            i = parent
            parent = self.parent(i)

    def parent(self, i: int) -> int:
        res = (i >> 1) - 1
        if res < 0:
            return 0
        else:
            return res

    def iter_heapify(self, i: int) -> None:
        while True:
            l: int = (i << 1) + 1
            r: int = (i << 1) + 2
            largest: int = i

            if l < self.size and self.cache[l] < self.cache[largest]:
                largest = l

            if r < self.size and self.cache[r] < self.cache[largest]:
                largest = r

            if largest == i:
                break
            else:
                self.cache[i], self.cache[largest] = self.cache[largest], self.cache[i]
                i = largest

In [10]:
%%timeit
def quick_sort(a: list[int], p: int=0, r: Union[int, None]=None):
    if r == None:
        r = len(a) - 1

    if p < r:
        q = partition(a, p, r)
        quick_sort(a, p, q-1)
        quick_sort(a, q+1, r)

    return a

749 ns ± 8.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
def partition(a: list[int], p: int, r: int) -> int:
    pivot: int = a[r]
    i = p

    for j in range(p, r):
        if a[j] <= pivot:
            a[i], a[j] = a[j], a[i]
            i += 1

    a[i], a[r] = a[r], a[i]

    return i

In [12]:
%%timeit
def heap_sort(a: list[int]) -> list[int]:
    n = len(a)

    for i in range((n >> 1) - 1, -1, -1):
        iter_heapify(a, i, n)

    for heap_size in range(n-1, 0, -1):
        a[0], a[heap_size] = a[heap_size], a[0]
        iter_heapify(a, 0, heap_size)

    return a

517 ns ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
def iter_heapify(a: list[int], i: int, length: int) -> None:
    while True:
        l: int = (i << 1) + 1
        r: int = (i << 1) + 2
        largest: int = i

        if l < length and a[l] > a[largest]:
            largest = l

        if r < length and a[r] > a[largest]:
            largest = r

        if largest == i:
            break
        else:
            a[i], a[largest] = a[largest], a[i]
            i = largest

In [14]:
%%timeit
def python_sort(a: list[int]) -> list[int]:
    a.sort()
    return a

522 ns ± 16.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
def main() -> None:
    sorts_list: list[Callable] = list([
        selection_sort,
        insertion_sort,
        stooge_sort,
        shell_sort,
        tournament_sort,
        quick_sort,
        heap_sort
    ])

    for sort_func in sorts_list:
        print(f"Testing sort: {sort_func.__name__}")
        for row in matrix_generator():
            a: list[int] = list(x for x in row)
            a_copy: list[int] = a.copy()

            undertest = sort_func(a_copy)
            expected = python_sort(a)

            assert expected == undertest, f"Sort {sort_func.__name__} is wrong"

        print("Good!")

In [16]:
if __name__ == "__main__":
    main()

NameError: name 'selection_sort' is not defined

## Среднее время выполнения


| Сортировка | Время |
-------- | ----- |
Стандартная функция | 324 ns ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)|
Выбором | 535 ns ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Вставкой | 507 ns ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Обменом | 1.01 µs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Шелла | 512 ns ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Турнирная | 536 ns ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Быстрая | 425 ns ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |
Пирамидальная | 518 ns ± 33.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each) |


Результаты получены при сортировке списка длиной 50 элементов в диапазоне [-250:1019]

Время работы получилось завышенным по причине того, что magic функция **timeit** не способна корректно замерять время у кода,
в котором используются type hintings